In [1]:
import types
from collections import Counter

from qiime.sdk import Artifact
from qiime.plugins import feature_classifier
import pandas as pd
from sklearn import (pipeline, naive_bayes, feature_selection, 
                     grid_search, cross_validation, metrics)
import skbio

from q2_feature_classifier._skl import _extract_labels, _extract_features

In [3]:
reference = Artifact.load('99_ref_feat_UNALIGNED.qza')

In [4]:
read_length = 75 # need to adjust for actual MOCK-6 data
f_primer = 'GTGCCAGCMGCCGCGGTAA' # from mockrobiota/data/mock-6/sample-metadata.tsv
r_primer = 'GGACTACHVGGGTWTCTAAT'
extract_reads = feature_classifier.methods.extract_paired_end_reads
reads = extract_reads(reference, read_length, f_primer, r_primer, method='match')

In [5]:
def get_seq_id(read):
    if isinstance(read, skbio.DNA):
        return read.metadata['id']
    else:
        return read[0].metadata['id']

In [6]:
word_length = 8
taxonomy_separator = '; '
taxonomy_depth = 6
multioutput = False
cv = 2
read_seqs = list(reads.view(types.GeneratorType))
taxonomy = reference.view(pd.Series)

seq_ids = [get_seq_id(s) for s in read_seqs]
labels = [taxonomy.get(s, 'unknown') for s in seq_ids]
labels = _extract_labels(labels, taxonomy_separator, taxonomy_depth, multioutput)
counted_labels = Counter(labels)
ok_labels = {l for l in counted_labels if counted_labels[l] >= 2*cv}

filtered = [(s, l) for s, l in zip(read_seqs, labels) if l in ok_labels]
read_seqs, y = zip(*filtered)
dummy, X = _extract_features(read_seqs, word_length)

# hold some back for validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.5, random_state=0, stratify=y)

In [7]:
classifier = naive_bayes.MultinomialNB()
selector = feature_selection.SelectPercentile()
steps = [('sel', selector), ('cls', classifier)]
pipeline = pipeline.Pipeline(steps)
grid_params = {'cls__alpha': [1., 0.01, 0.001],
               'sel__percentile': [100, 10, 1]}
grid = grid_search.GridSearchCV(pipeline, grid_params, cv=cv, n_jobs=4)
grid.fit(X_train, y_train)

/Users/ben/miniconda2/envs/qiime2/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)
/Users/ben/miniconda2/envs/qiime2/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)
/Users/ben/miniconda2/envs/qiime2/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)
/Users/ben/miniconda2/envs/qiime2/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)
/Users/ben/miniconda2/envs/qiime2/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)
/Users/ben/miniconda2/envs/qiime2/lib/python3.5/site-packages/skl

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(steps=[('sel', SelectPercentile(percentile=10,
         score_func=<function f_classif at 0x11156c400>)), ('cls', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'cls__alpha': [1.0, 0.01, 0.001], 'sel__percentile': [100, 10, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [8]:
print(grid.best_params_)

{'cls__alpha': 0.01, 'sel__percentile': 100}


In [9]:
y_true, y_pred = y_test, grid.predict(X_test)
report = metrics.classification_report(y_true, y_pred).split('\n')
print(report[0])
print(report[-2])

             precision    recall  f1-score   support
avg / total       0.83      0.83      0.82     83085


/Users/ben/miniconda2/envs/qiime2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
opposite_params = {'cls__alpha': 1., 'sel__percentile': 1}
pipeline.set_params(**opposite_params)
pipeline.fit(X_train, y_train)
y_true, y_pred = y_test, pipeline.predict(X_test)
report = metrics.classification_report(y_true, y_pred).split('\n')
print(report[0])
print(report[-2])

/Users/ben/miniconda2/envs/qiime2/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)


             precision    recall  f1-score   support
avg / total       0.37      0.45      0.36     83085


/Users/ben/miniconda2/envs/qiime2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
